# Metadata

**L1 Taxonomy** - Backend Integration

**L2 Taxonomy** - Webhooks

**Subtopic** - Transforming and storing webhook payloads into a database upon receipt

**Use Case** - Develop a Python module that listens for incoming webhook payloads, transforms the data, and stores it into a local JSON file acting as a database. The module should include functionality to parse the incoming JSON payload, transform it into a suitable format, and append it to the existing records in the local JSON file. Use Python's built-in json and http.server libraries for this task.

**Programming Language** - Python

**Target Model** - o1

# Setup

```requirements.txt
requests>=2.25.0
```


# Prompt
Problem Statement:
- Develop a Python module that listens for incoming webhook payloads, transforms the data, and stores it into a local JSON file acting as a database.
- The module should include functionality to parse the incoming JSON payload, transform it into a suitable format, and append it to the existing records in the local JSON file.
- Use Python's built-in json and http.server libraries for this task.
- Please make sure not to include unnecessary explanations or extra details. Just provide the complete implementation from start to end as per the prompt.

Input Format:
- An HTTP POST request is sent to the server's /webhook endpoint.
- The request body contains a valid JSON object with fields such as:
  - event_type: A string indicating the type of event (e.g., "user_signup").
  - timestamp: An ISO 8601 formatted string representing the event time (e.g., "2025-07-21T10:30:00Z").
  - user: A nested JSON object with user details (e.g., id, name, email).
  - metadata: (Optional) A nested JSON object with additional event metadata (e.g., source, campaign).

Output Format:
- The transformed JSON object is appended to a local JSON file named db.json.
- Each entry in db.json is stored as an element in a JSON array.
- The file grows with each valid incoming request and persists all previously received entries.
- Example entry after transformation:
  {
    "event_type": "user_signup",
    "timestamp": "2025-07-21T10:30:00Z",
    "user_id": "12345",
    "user_name": "Alice",
    "user_email": "alice@example.com",
    "source": "website",
    "campaign": "summer2025"
  }

Example:
```python
Input (HTTP POST JSON payload):
{
  "event_type": "user_signup",
  "timestamp": "2025-07-21T10:30:00Z",
  "user": {
    "id": "12345",
    "name": "Alice",
    "email": "alice@example.com"
  },
  "metadata": {
    "source": "website",
    "campaign": "summer2025"
  }
}

Transformed Output (appended to db.json):
{
  "event_type": "user_signup",
  "timestamp": "2025-07-21T10:30:00Z",
  "user_id": "12345",
  "user_name": "Alice",
  "user_email": "alice@example.com",
  "source": "website",
  "campaign": "summer2025"
}
```

Class Definition and Function Signature:
```python
class WebhookServer(http.server.BaseHTTPRequestHandler):
    """
    A simple HTTP server that listens for POST requests,
    transforms the incoming JSON payload, and stores it to a local JSON file.
    """

    def do_POST(self):
        """
        Handles HTTP POST requests:
        - Parses the JSON payload
        - Transforms it
        - Appends it to a local JSON file (db.json)
        """
        pass
```

# Requirements
Explicit Requirements:
- Use Python's built-in http.server to listen for HTTP POST requests.
- Use the built-in json module to parse and store JSON data.
- Accept a JSON payload via a POST request.
- Transform the payload into a suitable flat format.
- Append the transformed data to a local JSON file (db.json).
- Persist all received data across requests.
- Fields from nested dictionaries (user, metadata) should be flattened using underscore keys (e.g., user_id, metadata_source).

Implicit Requirements:
- The server should run continuously and handle multiple POST requests.
- The input JSON must be validated before transformation.
- The local file (db.json) must be created if it does not exist.
- File I/O should preserve existing data and not overwrite it.
- Handle errors gracefully (e.g., malformed JSON, I/O exceptions).
- Flatten nested structures like `user` and `metadata` into top-level fields.

Solution Expectations:
- A Python script that starts an HTTP server using http.server.
- The server accepts POST requests with a JSON body at a specific endpoint (e.g., /webhook).
- Incoming JSON payloads are parsed and validated.
- Nested fields (e.g., user, metadata) are flattened into a single-level dictionary.
- Transformed data is appended to a local JSON file named db.json.
- The JSON file maintains a list of all received entries.
- Existing data in db.json is preserved between runs.
- Proper error handling is implemented for malformed input and file operations.
- The script uses only Python's built-in libraries (no external dependencies).
- If optional fields are missing, they should be omitted from the transformed output (not set to null or defaulted).
- The handler must check that self.path == "/webhook" before processing POST requests.
- To handle concurrent requests, use ThreadingHTTPServer.
- When using ThreadingHTTPServer , file writes to shared resources like db.json must use explicit file locking to prevent race conditions and data corruption.

Edge Cases and Behavior:
- Empty JSON Body:
  - Behavior: Respond with 400 Bad Request and do not write to db.json.

- Malformed JSON Payload:
  - Behavior: Respond with 400 Bad Request and log an error.

- Missing Required Fields (e.g., event_type, timestamp, user):
  - Behavior: Respond with 400 Bad Request and provide a clear error message.

- Nested Fields are Deeper Than Expected:
  - Behavior: Either flatten known nested fields or ignore unknown depth beyond the expected structure.

- File db.json Does Not Exist:
  - Behavior: Create the file and initialize it as a JSON array.

- File db.json Is Empty or Corrupted:
  - Behavior: Reset the file with a fresh JSON array and log a warning.
  - If db.json is corrupted, rename it with a timestamp suffix (e.g., db_corrupted_2025-07-21.json) before creating a new one.

- Unsupported HTTP Method (e.g., GET, PUT):
  - Behavior: Respond with 405 Method Not Allowed.

- Repeated Identical Payloads:
  - Behavior: Append each one; deduplication is not required unless specified.

Solution Constraints:
- Must use only Python standard libraries (http.server, json, os, etc.).
- The server should accept only HTTP POST requests with a JSON body.
- Transformed data must be stored in a local file named db.json.
- db.json must store all entries as a JSON array (list of objects).
- The solution should not overwrite existing data in db.json.
- Nested structures (like "user" and "metadata") must be flattened into top-level fields.
- Maximum expected payload size should be reasonably handled (e.g., <1MB).
- The server should return appropriate HTTP status codes (e.g., 200, 400, 405).
- Code must follow clean structure with proper docstrings and comments.

In [ ]:
# code
"""Webhook listener server that receives JSON POSTs and stores events."""

import http.server
import socketserver
import json
import os
import threading
import time
import re

DB_FILE = "db.json"
DB_LOCK = threading.Lock()
ISO_8601_REGEX = r"^\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}(?:\.\d+)?Z$"


class WebhookServer(http.server.BaseHTTPRequestHandler):
    """
    A simple HTTP server that listens for POST requests.

    Transforms the incoming JSON payload and stores it to a local JSON file.
    """

    def _send_response(self, status_code, message):
        """Send a JSON response with the given status and message."""
        self.send_response(status_code)
        self.send_header("Content-type", "application/json")
        self.end_headers()
        self.wfile.write(json.dumps({"message": message}).encode("utf-8"))

    def _load_db(self):
        """Safely load JSON array from db.json, handling corruption."""
        with DB_LOCK:
            if not os.path.exists(DB_FILE):
                with open(DB_FILE, "w", encoding="utf-8") as f:
                    json.dump([], f)
                return []

            try:
                with open(DB_FILE, "r", encoding="utf-8") as f:
                    data = json.load(f)
                    if not isinstance(data, list):
                        raise ValueError("DB file content"
                                         " is not a JSON array.")
                    return data
            except (json.JSONDecodeError, ValueError):
                timestamp = time.strftime("%Y%m%d%H%M%S")
                corrupted_file = f"{DB_FILE}_corrupted_{timestamp}.json"
                os.rename(DB_FILE, corrupted_file)
                print(
                    f"Warning: db.json corrupted."
                    f" Renamed to {corrupted_file}. "
                    "Creating a new empty db.json."
                )
                with open(DB_FILE, "w", encoding="utf-8") as f:
                    json.dump([], f)
                return []

    def _save_db(self, data):
        """Write the given JSON array to db.json safely."""
        with DB_LOCK:
            with open(DB_FILE, "w", encoding="utf-8") as f:
                json.dump(data, f, indent=2)

    def do_POST(self):
        """
        Handle POST /webhook requests.

        Validates JSON payload and appends flattened version to db.json.
        """
        if self.path != "/webhook":
            self._send_response(404, "Not Found")
            return

        try:
            content_length = int(self.headers["Content-Length"])
            if content_length == 0:
                self._send_response(400, "Empty JSON Body")
                return

            post_body = self.rfile.read(content_length).decode("utf-8")
            payload = json.loads(post_body)
        except (TypeError, ValueError, json.JSONDecodeError):
            self._send_response(400,
                                "Malformed JSON Payload")
            return
        except Exception as exc:
            self._send_response(500,
                                f"Error reading request body: {exc}")
            return

        required_fields = ["event_type", "timestamp", "user"]
        if not all(field in payload for field in required_fields):
            self._send_response(400,
                                "Missing required top-level fields")
            return

        timestamp = payload.get("timestamp")
        if (not isinstance(timestamp, str) or
                not re.fullmatch(ISO_8601_REGEX, timestamp)):
            self._send_response(400,
                                "Invalid 'timestamp' format"
                                " (must be ISO 8601)")
            return

        user_data = payload.get("user")
        if not isinstance(user_data, dict):
            self._send_response(400,
                                "Invalid 'user' field: must be an object.")
            return

        for field in ["id", "name", "email"]:
            value = user_data.get(field)
            if not isinstance(value, str) or not value:
                self._send_response(400,
                                    f"Invalid 'user.{field}':"
                                    f" must be a non-empty string.")
                return

        if "@" not in user_data["email"] or "." not in user_data["email"]:
            self._send_response(400,
                                "Invalid 'user.email':"
                                " must be a valid email format.")
            return

        transformed_data = {
            "event_type": payload["event_type"],
            "timestamp": payload["timestamp"]
        }

        for key, value in user_data.items():
            transformed_data[f"user_{key}"] = value

        metadata = payload.get("metadata")
        if isinstance(metadata, dict):
            for key, value in metadata.items():
                transformed_data[key] = value

        try:
            db_data = self._load_db()
            db_data.append(transformed_data)
            self._save_db(db_data)
            self._send_response(200,
                                "Webhook received and processed successfully")
        except Exception as exc:
            self._send_response(500, f"Error processing webhook: {exc}")

    def do_GET(self):
        """Reject GET requests."""
        self._send_response(405, "Method Not Allowed")

    def do_PUT(self):
        """Reject PUT requests."""
        self._send_response(405, "Method Not Allowed")

    def do_DELETE(self):
        """Reject DELETE requests."""
        self._send_response(405, "Method Not Allowed")


class ThreadingHTTPServer(socketserver.ThreadingMixIn, http.server.HTTPServer):
    """Handle requests in a separate thread."""

    pass


def run_server(port=8000):
    """Start the HTTP webhook server on the given port."""
    server_address = ("", port)
    httpd = ThreadingHTTPServer(server_address, WebhookServer)
    print(f"Starting webhook server on port {port}...")
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:
        print("\nShutting down server.")
        httpd.shutdown()
        httpd.server_close()


if __name__ == "__main__":
    if not os.path.exists(DB_FILE):
        with open(DB_FILE, "w", encoding="utf-8") as f:
            json.dump([], f)
    else:
        try:
            with open(DB_FILE, "r", encoding="utf-8") as f:
                data = json.load(f)
                if not isinstance(data, list):
                    raise ValueError("DB file is not a JSON array.")
        except (json.JSONDecodeError, ValueError):
            timestamp = time.strftime("%Y%m%d%H%M%S")
            corrupted_file = f"{DB_FILE}_corrupted_{timestamp}.json"
            os.rename(DB_FILE, corrupted_file)
            print(
                f"Warning: db.json corrupted on startup. "
                f"Renamed to {corrupted_file}. Creating new empty db.json."
            )
            with open(DB_FILE, "w", encoding="utf-8") as f:
                json.dump([], f)

    run_server()


In [ ]:
# tests

"""Test module for HTTP server webhook JSON database functionality."""

import unittest
import json
import os
import tempfile
import shutil
from http.server import ThreadingHTTPServer
from threading import Thread
import time
import requests

from main import WebhookServer


class TestWebhookServer(unittest.TestCase):
    """Test suite for WebhookServer HTTP server functionality."""

    def setUp(self):
        """Set up test environment and start server for each test."""
        self.test_dir = tempfile.mkdtemp()
        self.original_dir = os.getcwd()
        os.chdir(self.test_dir)

        self.server = ThreadingHTTPServer(("localhost", 0), WebhookServer)
        self.server_thread = Thread(target=self.server.serve_forever)
        self.server_thread.daemon = True
        self.server_thread.start()
        time.sleep(0.5)

        port = self.server.server_address[1]
        self.base_url = f"http://localhost:{port}"

        if os.path.exists("db.json"):
            os.remove("db.json")

    def tearDown(self):
        """Stop server and clean up after each test."""
        self.server.shutdown()
        self.server_thread.join(timeout=2)

        os.chdir(self.original_dir)
        shutil.rmtree(self.test_dir)

    def test_successful_complete_payload(self):
        """Test webhook with all fields including optional metadata."""
        payload = {
            "event_type": "purchase",
            "timestamp": "2025-07-21T15:45:00Z",
            "user": {
                "id": "98765",
                "name": "Bob Smith",
                "email": "bob@example.com",
            },
            "metadata": {"source": "mobile_app", "campaign": "flash_sale"},
        }

        response = requests.post(f"{self.base_url}/webhook", json=payload)

        self.assertEqual(response.status_code, 200)

        with open("db.json", "r") as f:
            data = json.load(f)

        self.assertEqual(len(data), 1)
        entry = data[0]

        self.assertEqual(entry["event_type"], "purchase")
        self.assertEqual(entry["timestamp"], "2025-07-21T15:45:00Z")
        self.assertEqual(entry["user_id"], "98765")
        self.assertEqual(entry["user_name"], "Bob Smith")
        self.assertEqual(entry["user_email"], "bob@example.com")
        self.assertEqual(entry["source"], "mobile_app")
        self.assertEqual(entry["campaign"], "flash_sale")

    def test_only_required_fields(self):
        """Test webhook without optional metadata."""
        payload = {
            "event_type": "login",
            "timestamp": "2025-07-21T16:00:00Z",
            "user": {
                "id": "55555",
                "name": "Charlie",
                "email": "charlie@test.com",
            },
        }

        response = requests.post(f"{self.base_url}/webhook", json=payload)

        self.assertEqual(response.status_code, 200)

        with open("db.json", "r") as f:
            data = json.load(f)

        self.assertEqual(len(data), 1)
        entry = data[0]

        self.assertEqual(entry["event_type"], "login")
        self.assertEqual(entry["user_id"], "55555")

        self.assertNotIn("source", entry)
        self.assertNotIn("campaign", entry)

    def test_missing_timestamp(self):
        """Test validation when timestamp is missing."""
        payload = {
            "event_type": "logout",
            "user": {
                "id": "12345",
                "name": "David",
                "email": "david@example.com",
            },
        }

        response = requests.post(f"{self.base_url}/webhook", json=payload)

        self.assertEqual(response.status_code, 400)

        self.assertFalse(os.path.exists("db.json"))

    def test_empty_json_body(self):
        """Test empty request body."""
        response = requests.post(f"{self.base_url}/webhook", json={})

        self.assertEqual(response.status_code, 400)
        self.assertFalse(os.path.exists("db.json"))

    def test_malformed_json(self):
        """Test invalid JSON syntax."""
        response = requests.post(
            f"{self.base_url}/webhook",
            data='{"event_type": "test", invalid json}',
            headers={"Content-Type": "application/json"},
        )

        self.assertEqual(response.status_code, 400)
        self.assertFalse(os.path.exists("db.json"))

    def test_wrong_endpoint(self):
        """Test POST to incorrect endpoint."""
        payload = {
            "event_type": "test",
            "timestamp": "2025-07-21T16:00:00Z",
            "user": {"id": "123", "name": "Test", "email": "test@test.com"},
        }

        response = requests.post(f"{self.base_url}/invalid", json=payload)

        self.assertEqual(response.status_code, 404)
        self.assertFalse(os.path.exists("db.json"))

    def test_unsupported_method(self):
        """Test GET request to webhook endpoint."""
        response = requests.get(f"{self.base_url}/webhook")

        self.assertEqual(response.status_code, 405)

    def test_concurrent_requests(self):
        """Test thread safety with simultaneous requests."""
        payload1 = {
            "event_type": "view",
            "timestamp": "2025-07-21T17:00:00Z",
            "user": {"id": "111", "name": "User1", "email": "user1@test.com"},
        }

        payload2 = {
            "event_type": "click",
            "timestamp": "2025-07-21T17:00:01Z",
            "user": {"id": "222", "name": "User2", "email": "user2@test.com"},
        }

        import concurrent.futures

        with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
            future1 = executor.submit(
                requests.post, f"{self.base_url}/webhook", json=payload1
            )
            future2 = executor.submit(
                requests.post, f"{self.base_url}/webhook", json=payload2
            )

            response1 = future1.result()
            response2 = future2.result()

        self.assertEqual(response1.status_code, 200)
        self.assertEqual(response2.status_code, 200)

        with open("db.json", "r") as f:
            data = json.load(f)

        self.assertEqual(len(data), 2)

        user_ids = [entry["user_id"] for entry in data]
        self.assertIn("111", user_ids)
        self.assertIn("222", user_ids)

    def test_special_characters(self):
        """Test handling of special characters in values."""
        payload = {
            "event_type": "message",
            "timestamp": "2025-07-21T18:00:00Z",
            "user": {
                "id": "999",
                "name": 'O\'Brien "The Great"',
                "email": "test@example.com",
            },
            "metadata": {"source": "web\napp", "campaign": "summer\\2025"},
        }

        response = requests.post(f"{self.base_url}/webhook", json=payload)

        self.assertEqual(response.status_code, 200)

        with open("db.json", "r") as f:
            data = json.load(f)

        entry = data[0]
        self.assertEqual(entry["user_name"], 'O\'Brien "The Great"')
        self.assertEqual(entry["source"], "web\napp")
        self.assertEqual(entry["campaign"], "summer\\2025")

    def test_file_creation(self):
        """Test db.json creation when it doesn't exist."""
        self.assertFalse(os.path.exists("db.json"))

        payload = {
            "event_type": "first_event",
            "timestamp": "2025-07-21T19:00:00Z",
            "user": {
                "id": "001",
                "name": "First User",
                "email": "first@test.com",
            },
        }

        response = requests.post(f"{self.base_url}/webhook", json=payload)

        self.assertEqual(response.status_code, 200)
        self.assertTrue(os.path.exists("db.json"))

        with open("db.json", "r") as f:
            data = json.load(f)

        self.assertEqual(len(data), 1)

    def test_missing_event_type(self):
        """Test validation when event_type is missing."""
        payload = {
            "timestamp": "2025-07-21T23:30:00Z",
            "user": {
                "id": "444",
                "name": "Test User",
                "email": "test@example.com",
            },
        }

        response = requests.post(f"{self.base_url}/webhook", json=payload)

        self.assertEqual(response.status_code, 400)
        self.assertFalse(os.path.exists("db.json"))

    def test_missing_user_field(self):
        """Test validation when user field is missing."""
        payload = {
            "event_type": "invalid",
            "timestamp": "2025-07-21T23:45:00Z",
        }

        response = requests.post(f"{self.base_url}/webhook", json=payload)

        self.assertEqual(response.status_code, 400)
        self.assertFalse(os.path.exists("db.json"))

    def test_empty_db_file(self):
        """Test handling of empty (but existing) db.json."""
        open("db.json", "w").close()

        payload = {
            "event_type": "recovery",
            "timestamp": "2025-07-22T00:00:00Z",
            "user": {
                "id": "333",
                "name": "Recovery User",
                "email": "recovery@test.com",
            },
        }

        response = requests.post(f"{self.base_url}/webhook", json=payload)

        self.assertEqual(response.status_code, 200)

        with open("db.json", "r") as f:
            data = json.load(f)

        self.assertEqual(len(data), 1)
        self.assertEqual(data[0]["user_id"], "333")

    def test_deeply_nested_fields(self):
        """Test handling of unexpectedly deep nesting."""
        payload = {
            "event_type": "nested",
            "timestamp": "2025-07-22T00:15:00Z",
            "user": {
                "id": "nested123",
                "name": "Nested User",
                "email": "nested@test.com",
                "profile": {"settings": {"theme": "dark"}},
            },
        }

        response = requests.post(f"{self.base_url}/webhook", json=payload)

        self.assertEqual(response.status_code, 200)

        with open("db.json", "r") as f:
            data = json.load(f)

        entry = data[0]
        self.assertEqual(entry["user_id"], "nested123")
        self.assertEqual(entry["user_name"], "Nested User")
        self.assertEqual(entry["user_email"], "nested@test.com")

        if "user_profile" in entry:
            self.assertIsInstance(entry["user_profile"], dict)
            self.assertEqual(
                entry["user_profile"]["settings"]["theme"], "dark"
            )
        else:
            self.assertNotIn("theme", entry)
            self.assertNotIn("settings", entry)

    def test_put_request(self):
        """Test PUT request to webhook endpoint."""
        payload = {
            "event_type": "test",
            "timestamp": "2025-07-22T00:30:00Z",
            "user": {"id": "123", "name": "Test", "email": "test@test.com"},
        }

        response = requests.put(f"{self.base_url}/webhook", json=payload)

        self.assertEqual(response.status_code, 405)
        self.assertFalse(os.path.exists("db.json"))


if __name__ == "__main__":
    unittest.main()


# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/40fca8d9-4e2b-41be-b38d-117d7fdd3af7?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F81109%2Fview

#### Model code:

```python
# code generated by the model
#!/usr/bin/env python3

import os
import time
import json
from threading import Lock
from http.server import BaseHTTPRequestHandler, ThreadingHTTPServer

DB_FILE = 'db.json'
db_lock = Lock()

def init_db():
    """
    Initializes or repairs the local JSON file (db.json).
    If db.json does not exist, creates an empty JSON list.
    If db.json is corrupted, renames it and creates a fresh file.
    """
    if not os.path.exists(DB_FILE):
        with open(DB_FILE, 'w', encoding='utf-8') as f:
            json.dump([], f)
        return
    try:
        with open(DB_FILE, 'r', encoding='utf-8') as f:
            data = json.load(f)
            if not isinstance(data, list):
                raise ValueError("db.json is not a list.")
    except Exception:
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        os.rename(DB_FILE, f"db_corrupted_{timestamp}.json")
        with open(DB_FILE, 'w', encoding='utf-8') as f:
            json.dump([], f)

def flatten_payload(payload):
    """
    Flattens the incoming JSON payload.
    Required top-level fields: event_type, timestamp, user.
    Optional nested metadata fields: source, campaign.
    """
    if not all(k in payload for k in ("event_type", "timestamp", "user")):
        raise ValueError("Missing required fields in JSON payload.")
    if not isinstance(payload["user"], dict):
        raise ValueError("User field must be a JSON object.")
    
    # <Issue>: Missing validation for required user subfields (id, name, email)
    flattened = {
        "event_type": payload["event_type"],
        "timestamp": payload["timestamp"],
        "user_id": payload["user"].get("id", ""),# <Issue>: Should raise error if missing
        "user_name": payload["user"].get("name", ""),
        "user_email": payload["user"].get("email", "")
    }
    # <Issue>: No validation for ISO 8601 format on timestamp
    metadata = payload.get("metadata", {})
    if isinstance(metadata, dict):
        if "source" in metadata:
            flattened["source"] = metadata["source"]
        if "campaign" in metadata:
            flattened["campaign"] = metadata["campaign"]

    return flattened

class WebhookServer(BaseHTTPRequestHandler):
    """
    A simple HTTP server that listens for POST /webhook requests,
    transforms the incoming JSON payload, and stores it to db.json.
    """

    def do_GET(self):
        self.send_response(405)
        self.end_headers()

    def do_PUT(self):
        self.send_response(405)
        self.end_headers()

    def do_DELETE(self):
        self.send_response(405)
        self.end_headers()

    def do_POST(self):
        if self.path != "/webhook":
            self.send_response(404)
            self.end_headers()
            return

        content_length = self.headers.get('Content-Length')
        if not content_length:
            self.send_response(400)
            self.end_headers()
            return

        try:
            length = int(content_length)
            payload_data = self.rfile.read(length)
            payload_json = json.loads(payload_data.decode('utf-8'))
        except Exception:
            self.send_response(400)
            self.end_headers()
            return

        try:
            flattened = flatten_payload(payload_json)
        except ValueError as e:
            self.send_response(400)
            self.end_headers()
            return

        with db_lock:
            # Read current data
            try:
                with open(DB_FILE, 'r', encoding='utf-8') as f:
                    existing_data = json.load(f)
                    if not isinstance(existing_data, list):
                        existing_data = []
            except Exception:
                existing_data = []

            # Append new entry
            existing_data.append(flattened)

            # Write back to file
            with open(DB_FILE, 'w', encoding='utf-8') as f:
                json.dump(existing_data, f, indent=2)

        self.send_response(200)
        self.end_headers()
        # <Issue>: No response body sent for success or errors

def main():
    init_db()
    server = ThreadingHTTPServer(('0.0.0.0', 8080), WebhookServer)
    server.serve_forever()

if __name__ == '__main__':
    main()
```